In [1]:
import numpy as np
import pandas as pd
import geopy.distance
import haversine as hs
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
df = pd.read_csv('../Base/Base_raio_grandefortaleza_230205.csv')
df

,Código Cliente,Cliente,HC Rota,Rota,Cód Produto,Produto,Volume,Faturamento Total,Quantidade Unidade,Categoria Comercial,...,Subcategoria,Bairro,RUA,DESCRICAO_CIDADE,Ano/Mês,Descrição Segmentacao,Código Segmentação,Cliente lat,Cliente long,Tipo Pedido
0,1024744,ANCORA DISTRIBUIDORA DE ALIMENTOS L,1501343,CECAS006,401060,LEITE UHT INT EDGE BETA 1L,195840.00,854352,195840.0,LEITE UHT INT EDGE,...,LEITE UHT INTEGRAL,DT INDUSTRIAL,RUA LESTE 3 289,MARACANAÚ,2022/12,KEY ACCOUNT REGIONAL,63,-3.863180,-38.599760,Faturamento - Venda
1,1024744,ANCORA DISTRIBUIDORA DE ALIMENTOS L,1501343,CECAS006,401060,LEITE UHT INT EDGE BETA 1L,171360.00,753984,171360.0,LEITE UHT INT EDGE,...,LEITE UHT INTEGRAL,DT INDUSTRIAL,RUA LESTE 3 289,MARACANAÚ,2022/11,KEY ACCOUNT REGIONAL,63,-3.863180,-38.599760,Faturamento - Venda
2,1024744,ANCORA DISTRIBUIDORA DE ALIMENTOS L,1501343,CECAS006,401060,LEITE UHT INT EDGE BETA 1L,155040.00,666672,155040.0,LEITE UHT INT EDGE,...,LEITE UHT INTEGRAL,DT INDUSTRIAL,RUA LESTE 3 289,MARACANAÚ,2023/01,KEY ACCOUNT REGIONAL,63,-3.863180,-38.599760,Faturamento - Venda
3,1095987,SUPERMERCADO COMETA EIRELI,1501339,CECAS002,401060,LEITE UHT INT EDGE BETA 1L,122400.00,521424,122400.0,LEITE UHT INT EDGE,...,LEITE UHT INTEGRAL,PLANALTO AYRTON SENNA,AV BERNARDO MANUEL 14354,FORTALEZA,2023/01,KEY ACCOUNT REGIONAL,63,-3.843301,-38.565730,Faturamento - Venda
4,1095987,SUPERMERCADO COMETA EIRELI,1501339,CECAS002,400048,LEITE UHT INT JAGUA 1L,103680.00,456192,103680.0,LEITE UHT INT,...,LEITE UHT INTEGRAL,PLANALTO AYRTON SENNA,AV BERNARDO MANUEL 14354,FORTALEZA,2022/11,KEY ACCOUNT REGIONAL,63,-3.843301,-38.565730,Faturamento - Venda
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257365,1013880,PAJUCARA DIST DE ALIMENTOS LTDA,1501344,CECAT007,401060,LEITE UHT INT EDGE BETA 1L,-2400.00,-10320,-2400.0,LEITE UHT INT EDGE,...,LEITE UHT INTEGRAL,DT INDUST I,"ROD DR MENDEL STEINBRUCH SN, LJ19",MARACANAÚ,2023/01,OUTROS,66,-3.804471,-38.572465,Devolução
257366,1035951,ATACADAO SA,1501646,CECKA001,400038,CREME DE LEITE BETA 200G,-3159.00,"-39487,5",-15795.0,CREME DE LEITE,...,CREME DE LEITE,ITAMBE,RUAOESTRUTURANTE CE.85 S/N,CAUCAIA,2022/12,CASH CARRY,58,-3.724081,-38.644575,Devolução
257367,1057624,MERCANTIL MEDEIROS LTDA,4014301,JOSE RIBAMAR CONSTRUCOES E REPRESEN,402430,LEITE UHT INT CAMPONESA EDGE 1L,-3600.00,-16560,-3600.0,LEITE UHT INT,...,LEITE UHT INTEGRAL,MONDUBIM,R ROQUE MEDEIROS 1151,FORTALEZA,2022/12,AUTO SERVIÇO DIRETO,59,-3.820944,-38.560742,Devolução
257368,1045514,ORGANIZACAO CEREALISTA SERRANO LTDA,1500592,CECVA001,402503,"BEB ADOCADA FRUTAS C TAMPICO 1,5L",-4469.76,"-16761,6",-2880.0,"TAMPICO 1,5L CIT",...,BEBIDA MISTA,CENTRO,"R GOVERNADOR SAMPAIO 395, 405 407 R",FORTALEZA,2023/01,OUTROS,66,-3.728829,-38.524093,Devolução


In [3]:
#
df_prod = df.rename(columns={'Código Cliente':'Cliente_cod'})
df_prod = df_prod.rename(columns={'Canal de Vendas':'Canal_de_vendas'})
df_prod = df_prod.rename(columns={'Tipo Pedido':'Tipo_Pedido'})
df_prod = df_prod.rename(columns={'Cliente lat':'Lat'})
df_prod = df_prod.rename(columns={'Cliente long':'Long'})
df_prod['Cliente_cod'] = df_prod['Cliente_cod'].astype(str)
df_prod = df_prod[(df_prod.Tipo_Pedido=='Faturamento - Venda')]
df_prod = df_prod[(df_prod.Canal_de_vendas=='Pequeno varejo') | (df_prod.Canal_de_vendas=='Atacado Regional')
                 | (df_prod.Canal_de_vendas=='Auto Serv. Regional') | (df_prod.Tipo_Pedido=='Faturamento - Venda')]
df_prod = df_prod[['Cliente_cod','Produto','Lat','Long']].drop_duplicates().reset_index(drop=True)
df_prod = df_prod.rename(columns={'Cliente_cod':'Cliente'})
df_prod

,Cliente,Produto,Lat,Long
0,1024744,LEITE UHT INT EDGE BETA 1L,-3.863180,-38.599760
1,1095987,LEITE UHT INT EDGE BETA 1L,-3.843301,-38.565730
2,1095987,LEITE UHT INT JAGUA 1L,-3.843301,-38.565730
3,1060211,CREME DE LEITE CAMPONESA 17% UHT 200G,-3.869474,-38.509369
4,1042440,LEITE UHT INT EDGE BETA 1L,-3.776480,-38.602625
...,...,...,...,...
100424,1084536,IOG INT BANANA E MACA BETANIA KIDS 100G,-3.733134,-38.501440
100425,1143527,IOG INT BANANA E MACA BETANIA KIDS 100G,-3.872350,-38.449072
100426,1013078,IOG INT BANANA E MACA BETANIA KIDS 100G,-3.848111,-38.393247
100427,1130523,IOG INT MORANGO BET KIDS MERENDINHA 100G,-4.142762,-38.756499


In [4]:
df_final = pd.get_dummies(df_prod, columns=['Produto'], prefix='', prefix_sep='').reset_index(drop=True)
df_final = df_final.groupby(['Cliente','Lat','Long']).sum().reset_index()
df_final

,Cliente,Lat,Long,"BEB ADOCADA FRUTAS C TAMPICO 1,5L",BEB DE FRUTA ADOCADA FRUT CIT TAMP 330ML,BEB DE FRUTA ADOCADA UVA TAMP 2L,BEB DE FRUTA ADOCADA UVA TAMP 330ML,BEB LAC COM PROTEINA BANANA,BEB LAC COM PROTEINA CHOCOLATE,BEB LAC UHT CHOCOLATE BETANIA KIDS 200ML,...,SOBREMESA FLAN CALDA CARAMELO BETA 220G,TAMPICO FRUTAS CITRICAS 200ML,TAMPICO UVA 200ML,VARREDURA LEITE EM PO,YOBEM TRIPLO ZERO AMEIXA 170G,YOBEM TRIPLO ZERO AMEIXA 850G,YOBEM TRIPLO ZERO MORANGO 170G,YOBEM TRIPLO ZERO MORANGO 850G,YOBEM TRIPLO ZERO PAPAIA C/ LIN 170G,YOBEM TRIPLO ZERO PAPAIA C/ LIN 850G
0,1000045,-3.818212,-38.534009,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1000048,-3.763059,-38.531750,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1000052,-3.739469,-38.476386,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1000062,-3.747308,-38.553298,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1000077,-3.790181,-38.472887,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5513,1251205,-3.812659,-38.527400,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5514,2000198,-3.745475,-38.507828,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5515,2000199,-3.730466,-38.496918,0,0,0,0,1,1,1,...,0,0,0,0,1,0,1,0,1,0
5516,2001705,-3.801106,-38.577521,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [5]:
final_temp = pd.Series(dtype='int64')
final_end = pd.DataFrame()
df_final_stage = df_final.drop_duplicates()

for i in range(len(df_final_stage)):   
    final = pd.Series(dtype='int64')
    distancia = []
    df_final_stage = df_final.drop_duplicates()
    for j in range(len(df_final_stage)):
        
        loc1=(df_final_stage['Lat'].iloc[i], df_final_stage['Long'].iloc[i])
        loc2=(df_final_stage['Lat'].iloc[j], df_final_stage['Long'].iloc[j])
        distancia.append(hs.haversine(loc1,loc2))   
        
    df_final_stage[df_final_stage['Cliente'].iloc[i]+"_dist"] = distancia   
 
    coluna = df_final_stage['Cliente'].iloc[i]+"_dist"
    
    cliente = df_final_stage['Cliente'].iloc[i]
    lat = df_final_stage['Lat'].iloc[i]
    long = df_final_stage['Long'].iloc[i]
    
    
    
    df_final1 = df_final_stage.loc[df_final_stage[coluna] < 1].reset_index(drop=True)    
    
    
    
    num_cli = len(df_final1.index)

    sum = df_final1.sum()
    sum.name = 'Sum'
    
    sum = pd.DataFrame(sum)
    
    df_final2 = pd.concat([df_final1,sum.T], axis=0)
    
    df_final2 = df_final2.drop([coluna],axis=1)
    
    df_final2 = df_final2.drop_duplicates()
    
    cliente1 = df_final2[(df_final2['Cliente'] == cliente) & (df_final2['Lat'] == lat) & (df_final2['Long'] == long)]
    
    
    
    cliente1 = cliente1.reset_index(drop=True)
    cliente1 = cliente1*-999
    
    
    
    soma = df_final2.filter(items = ['Sum'], axis=0)
    
    
    soma = soma.reset_index(drop=True)
    
    df_final_comp = cliente1.add(soma)
    
    df_final_comp = df_final_comp.drop(['Cliente','Long','Lat'],axis=1)
    
    "df_final_comp = df_final_comp.div(num_cli-1)"
    
    
     
    "df_final_comp = df_final_comp.sub(num_cli-1)"
    
    resultados = df_final_comp.T
    
    "resultados = pd.concat([resultados,resultados], axis=1)"
    "resultados = resultados.rename(columns={0:cliente})"
    resultados = resultados.sort_values(by=[0], ascending=False)
    try:
        resultados = resultados.drop(columns=[1])
    except:
        pass
    final1 = resultados.iloc[0:10].index.to_series().reset_index(drop=True)
    final = pd.concat([final,final1], axis=1)
    final = final.reset_index(drop=True)
    try:
        final = final.drop(columns=[0])
        
    finally:
   
        resultados = resultados.reset_index(drop=True)
        resultados = resultados.squeeze()
        resultados = resultados.iloc[0:10]
        final_temp = pd.concat([final,resultados], axis=1).reset_index(drop=True)
        final_temp['Cliente'] = cliente
        final_temp['Total_pdv'] = num_cli
        final_end = final_end.reset_index(drop=True)
        final_temp = final_temp.reset_index(drop=True)
        try:
            final_end = pd.concat([final_end,final_temp.iloc[0:10]], axis=0)
        except:
            pass
        final_end = final_end.reset_index(drop=True)
        
        print(str(round(i/len(df_final_stage)*100,2))+"%")
        
        
final_end.rename(columns={ final_end.columns[0]: "Produto" }, inplace = True)
final_end.rename(columns={ final_end.columns[1]: "Presenca" }, inplace = True)

final_end

0.0%
0.02%
0.04%
0.05%
0.07%
0.09%
0.11%
0.13%
0.14%
0.16%
0.18%
0.2%
0.22%
0.24%
0.25%
0.27%
0.29%
0.31%
0.33%
0.34%
0.36%
0.38%
0.4%
0.42%
0.43%
0.45%
0.47%
0.49%
0.51%
0.53%
0.54%
0.56%
0.58%
0.6%
0.62%
0.63%
0.65%
0.67%
0.69%
0.71%
0.72%
0.74%
0.76%
0.78%
0.8%
0.82%
0.83%
0.85%
0.87%
0.89%
0.91%
0.92%
0.94%
0.96%
0.98%
1.0%
1.01%
1.03%
1.05%
1.07%
1.09%
1.11%
1.12%
1.14%
1.16%
1.18%
1.2%
1.21%
1.23%
1.25%
1.27%
1.29%
1.3%
1.32%
1.34%
1.36%
1.38%
1.4%
1.41%
1.43%
1.45%
1.47%
1.49%
1.5%
1.52%
1.54%
1.56%
1.58%
1.59%
1.61%
1.63%
1.65%
1.67%
1.69%
1.7%
1.72%
1.74%
1.76%
1.78%
1.79%
1.81%
1.83%
1.85%
1.87%
1.88%
1.9%
1.92%
1.94%
1.96%
1.98%
1.99%
2.01%
2.03%
2.05%
2.07%
2.08%
2.1%
2.12%
2.14%
2.16%
2.17%
2.19%
2.21%
2.23%
2.25%
2.27%
2.28%
2.3%
2.32%
2.34%
2.36%
2.37%
2.39%
2.41%
2.43%
2.45%
2.46%
2.48%
2.5%
2.52%
2.54%
2.56%
2.57%
2.59%
2.61%
2.63%
2.65%
2.66%
2.68%
2.7%
2.72%
2.74%
2.75%
2.77%
2.79%
2.81%
2.83%
2.85%
2.86%
2.88%
2.9%
2.92%
2.94%
2.95%
2.97%
2.99%
3.01%
3.03%
3.04%
3.0

22.98%
23.0%
23.02%
23.03%
23.05%
23.07%
23.09%
23.11%
23.12%
23.14%
23.16%
23.18%
23.2%
23.21%
23.23%
23.25%
23.27%
23.29%
23.31%
23.32%
23.34%
23.36%
23.38%
23.4%
23.41%
23.43%
23.45%
23.47%
23.49%
23.5%
23.52%
23.54%
23.56%
23.58%
23.6%
23.61%
23.63%
23.65%
23.67%
23.69%
23.7%
23.72%
23.74%
23.76%
23.78%
23.79%
23.81%
23.83%
23.85%
23.87%
23.89%
23.9%
23.92%
23.94%
23.96%
23.98%
23.99%
24.01%
24.03%
24.05%
24.07%
24.08%
24.1%
24.12%
24.14%
24.16%
24.18%
24.19%
24.21%
24.23%
24.25%
24.27%
24.28%
24.3%
24.32%
24.34%
24.36%
24.37%
24.39%
24.41%
24.43%
24.45%
24.47%
24.48%
24.5%
24.52%
24.54%
24.56%
24.57%
24.59%
24.61%
24.63%
24.65%
24.66%
24.68%
24.7%
24.72%
24.74%
24.76%
24.77%
24.79%
24.81%
24.83%
24.85%
24.86%
24.88%
24.9%
24.92%
24.94%
24.95%
24.97%
24.99%
25.01%
25.03%
25.05%
25.06%
25.08%
25.1%
25.12%
25.14%
25.15%
25.17%
25.19%
25.21%
25.23%
25.24%
25.26%
25.28%
25.3%
25.32%
25.34%
25.35%
25.37%
25.39%
25.41%
25.43%
25.44%
25.46%
25.48%
25.5%
25.52%
25.53%
25.55%
25.57%
25.59%


44.51%
44.53%
44.55%
44.56%
44.58%
44.6%
44.62%
44.64%
44.65%
44.67%
44.69%
44.71%
44.73%
44.74%
44.76%
44.78%
44.8%
44.82%
44.84%
44.85%
44.87%
44.89%
44.91%
44.93%
44.94%
44.96%
44.98%
45.0%
45.02%
45.03%
45.05%
45.07%
45.09%
45.11%
45.13%
45.14%
45.16%
45.18%
45.2%
45.22%
45.23%
45.25%
45.27%
45.29%
45.31%
45.32%
45.34%
45.36%
45.38%
45.4%
45.42%
45.43%
45.45%
45.47%
45.49%
45.51%
45.52%
45.54%
45.56%
45.58%
45.6%
45.61%
45.63%
45.65%
45.67%
45.69%
45.7%
45.72%
45.74%
45.76%
45.78%
45.8%
45.81%
45.83%
45.85%
45.87%
45.89%
45.9%
45.92%
45.94%
45.96%
45.98%
45.99%
46.01%
46.03%
46.05%
46.07%
46.09%
46.1%
46.12%
46.14%
46.16%
46.18%
46.19%
46.21%
46.23%
46.25%
46.27%
46.28%
46.3%
46.32%
46.34%
46.36%
46.38%
46.39%
46.41%
46.43%
46.45%
46.47%
46.48%
46.5%
46.52%
46.54%
46.56%
46.57%
46.59%
46.61%
46.63%
46.65%
46.67%
46.68%
46.7%
46.72%
46.74%
46.76%
46.77%
46.79%
46.81%
46.83%
46.85%
46.86%
46.88%
46.9%
46.92%
46.94%
46.96%
46.97%
46.99%
47.01%
47.03%
47.05%
47.06%
47.08%
47.1%
47.12%


66.04%
66.06%
66.07%
66.09%
66.11%
66.13%
66.15%
66.17%
66.18%
66.2%
66.22%
66.24%
66.26%
66.27%
66.29%
66.31%
66.33%
66.35%
66.36%
66.38%
66.4%
66.42%
66.44%
66.46%
66.47%
66.49%
66.51%
66.53%
66.55%
66.56%
66.58%
66.6%
66.62%
66.64%
66.65%
66.67%
66.69%
66.71%
66.73%
66.75%
66.76%
66.78%
66.8%
66.82%
66.84%
66.85%
66.87%
66.89%
66.91%
66.93%
66.94%
66.96%
66.98%
67.0%
67.02%
67.04%
67.05%
67.07%
67.09%
67.11%
67.13%
67.14%
67.16%
67.18%
67.2%
67.22%
67.23%
67.25%
67.27%
67.29%
67.31%
67.33%
67.34%
67.36%
67.38%
67.4%
67.42%
67.43%
67.45%
67.47%
67.49%
67.51%
67.52%
67.54%
67.56%
67.58%
67.6%
67.62%
67.63%
67.65%
67.67%
67.69%
67.71%
67.72%
67.74%
67.76%
67.78%
67.8%
67.81%
67.83%
67.85%
67.87%
67.89%
67.91%
67.92%
67.94%
67.96%
67.98%
68.0%
68.01%
68.03%
68.05%
68.07%
68.09%
68.1%
68.12%
68.14%
68.16%
68.18%
68.19%
68.21%
68.23%
68.25%
68.27%
68.29%
68.3%
68.32%
68.34%
68.36%
68.38%
68.39%
68.41%
68.43%
68.45%
68.47%
68.48%
68.5%
68.52%
68.54%
68.56%
68.58%
68.59%
68.61%
68.63%
68.65

87.57%
87.59%
87.6%
87.62%
87.64%
87.66%
87.68%
87.69%
87.71%
87.73%
87.75%
87.77%
87.79%
87.8%
87.82%
87.84%
87.86%
87.88%
87.89%
87.91%
87.93%
87.95%
87.97%
87.98%
88.0%
88.02%
88.04%
88.06%
88.08%
88.09%
88.11%
88.13%
88.15%
88.17%
88.18%
88.2%
88.22%
88.24%
88.26%
88.27%
88.29%
88.31%
88.33%
88.35%
88.37%
88.38%
88.4%
88.42%
88.44%
88.46%
88.47%
88.49%
88.51%
88.53%
88.55%
88.56%
88.58%
88.6%
88.62%
88.64%
88.66%
88.67%
88.69%
88.71%
88.73%
88.75%
88.76%
88.78%
88.8%
88.82%
88.84%
88.85%
88.87%
88.89%
88.91%
88.93%
88.95%
88.96%
88.98%
89.0%
89.02%
89.04%
89.05%
89.07%
89.09%
89.11%
89.13%
89.14%
89.16%
89.18%
89.2%
89.22%
89.24%
89.25%
89.27%
89.29%
89.31%
89.33%
89.34%
89.36%
89.38%
89.4%
89.42%
89.43%
89.45%
89.47%
89.49%
89.51%
89.53%
89.54%
89.56%
89.58%
89.6%
89.62%
89.63%
89.65%
89.67%
89.69%
89.71%
89.72%
89.74%
89.76%
89.78%
89.8%
89.82%
89.83%
89.85%
89.87%
89.89%
89.91%
89.92%
89.94%
89.96%
89.98%
90.0%
90.01%
90.03%
90.05%
90.07%
90.09%
90.11%
90.12%
90.14%
90.16%
90.18

,Produto,Presenca,Cliente,Total_pdv
0,LEITE UHT INT EDGE BETA 1L,19,1000045,24
1,REQUEIJAO CREMOSO BETA 200G,16,1000045,24
2,CREME DE LEITE BETA 200G,14,1000045,24
3,LEITE CONDENSADO BETA 395G,12,1000045,24
4,LEITE UHT DESNT EDGE BETA 1L,12,1000045,24
...,...,...,...,...
55175,BEB DE FRUTA ADOCADA FRUT CIT TAMP 330ML,26,2001766,80
55176,IOG GARR PARC DESN AMEIXA BETA 170G,24,2001766,80
55177,REQUEIJAO CREMOSO LIGHT BETA 200G,24,2001766,80
55178,IOG NATURAL INT BETA COPO 170G,23,2001766,80


In [19]:
final_end.to_excel("../Output/clusterização_tabela_1km_230205.xlsx")

In [6]:
clientes = df[['Código Cliente','Rota','Supervisor','Regional','Vendedor','RUA', 'Bairro', 'Descrição Segmentacao', 'Volume']]
clientes = clientes.groupby(['Código Cliente','Rota','Supervisor','Regional','Vendedor','RUA', 'Bairro', 'Descrição Segmentacao']).sum().reset_index()
clientes = clientes.sort_values(by=['Volume'], ascending=False)
clientes['Código Cliente'] = clientes['Código Cliente'].astype(str)
clientes = clientes.rename(columns={'Código Cliente':'Cliente'})
clientes = clientes.sort_values('Volume', ascending=False).drop_duplicates('Cliente').sort_index()
final_completa = final_end.merge(clientes, on='Cliente', how='left')

In [7]:
final_completa.to_excel("../Output/Base_raio_1km_grandefortaleza_230205.xlsx")